In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stockdata.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 17




#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 17





# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [2]:
#we store the variables here
log_files_path = 'C:/Users/jira_/Desktop/new_st22-15-22/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [22, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 22], [22], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '15'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 22]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/new_st22-15-22/tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,22)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.438568711
Validation Loss: 2.7951818
Epoch: 0002 cost = 2.423783898
Validation Loss: 2.582018
Epoch: 0003 cost = 2.408987363
Validation Loss: 2.4138298
Epoch: 0004 cost = 2.393864155
Validation Loss: 2.3217702
Epoch: 0005 cost = 2.377999942
Validation Loss: 2.26249
Epoch: 0006 cost = 2.361153563
Validation Loss: 2.198143
Epoch: 0007 cost = 2.343168497
Validation Loss: 2.1331975
Epoch: 0008 cost = 2.324018319
Validation Loss: 2.0750334
Epoch: 0009 cost = 2.304303010
Validation Loss: 2.0093892
Epoch: 0010 cost = 2.284662565
Validation Loss: 1.9758154
Epoch: 0011 cost = 2.265613397
Validation Loss: 1.9394817
Epoch: 0012 cost = 2.247368813
Validation Loss: 1.9062381
Epoch: 0013 cost = 2.229685267
Validation Loss: 1.8986285
Epoch: 0014 cost = 2.212151051
Validation Loss: 1.9179262
Epoch: 0015 cost = 2.194336931
Validation Loss: 1.9216291
Epoch: 0016 cost = 2.175695737
Validation Loss: 1.8988662
Epoch: 0017 cost = 2.156454086
Validation Loss: 1.9039356
Epoch: 00

Validation Loss: 1.144215
Epoch: 0156 cost = 1.163797736
Validation Loss: 1.1386125
Epoch: 0157 cost = 1.158976813
Validation Loss: 1.1472346
Epoch: 0158 cost = 1.154175162
Validation Loss: 1.1652116
Epoch: 0159 cost = 1.149392664
Validation Loss: 1.1481225
Epoch: 0160 cost = 1.144630333
Validation Loss: 1.1618512
Epoch: 0161 cost = 1.139886717
Validation Loss: 1.2085946
Epoch: 0162 cost = 1.135162115
Validation Loss: 1.1928238
Epoch: 0163 cost = 1.130456507
Validation Loss: 1.1863214
Epoch: 0164 cost = 1.125769754
Validation Loss: 1.1779168
Epoch: 0165 cost = 1.121101499
Validation Loss: 1.1159431
Epoch: 0166 cost = 1.116451820
Validation Loss: 1.0617617
Epoch: 0167 cost = 1.111820877
Validation Loss: 1.08542
Epoch: 0168 cost = 1.107208312
Validation Loss: 1.1111128
Epoch: 0169 cost = 1.102614045
Validation Loss: 1.1094257
Epoch: 0170 cost = 1.098037879
Validation Loss: 1.1206222
Epoch: 0171 cost = 1.093480488
Validation Loss: 1.0750321
Epoch: 0172 cost = 1.088940581
Validation Loss: 

Epoch: 0317 cost = 0.614595731
Validation Loss: 0.6734028
Epoch: 0318 cost = 0.612419774
Validation Loss: 0.6884698
Epoch: 0319 cost = 0.610254715
Validation Loss: 0.7021593
Epoch: 0320 cost = 0.608101507
Validation Loss: 0.69857395
Epoch: 0321 cost = 0.605959594
Validation Loss: 0.68592334
Epoch: 0322 cost = 0.603828490
Validation Loss: 0.652209
Epoch: 0323 cost = 0.601709445
Validation Loss: 0.63978785
Epoch: 0324 cost = 0.599600881
Validation Loss: 0.62654656
Epoch: 0325 cost = 0.597503781
Validation Loss: 0.62210774
Epoch: 0326 cost = 0.595417420
Validation Loss: 0.6332345
Epoch: 0327 cost = 0.593342543
Validation Loss: 0.62895906
Epoch: 0328 cost = 0.591277599
Validation Loss: 0.6208473
Epoch: 0329 cost = 0.589224150
Validation Loss: 0.6106724
Epoch: 0330 cost = 0.587182075
Validation Loss: 0.60971165
Epoch: 0331 cost = 0.585150808
Validation Loss: 0.61901027
Epoch: 0332 cost = 0.583130380
Validation Loss: 0.6053798
Epoch: 0333 cost = 0.581121037
Validation Loss: 0.5680936
Epoch: 

Epoch: 0472 cost = 0.385404567
Validation Loss: 0.41469282
Epoch: 0473 cost = 0.384434283
Validation Loss: 0.3984381
Epoch: 0474 cost = 0.383468469
Validation Loss: 0.4083509
Epoch: 0475 cost = 0.382507394
Validation Loss: 0.3929546
Epoch: 0476 cost = 0.381550744
Validation Loss: 0.40837312
Epoch: 0477 cost = 0.380598649
Validation Loss: 0.39623013
Epoch: 0478 cost = 0.379650791
Validation Loss: 0.39479384
Epoch: 0479 cost = 0.378707697
Validation Loss: 0.39573243
Epoch: 0480 cost = 0.377768492
Validation Loss: 0.3899933
Epoch: 0481 cost = 0.376833955
Validation Loss: 0.40110147
Epoch: 0482 cost = 0.375903845
Validation Loss: 0.38692564
Epoch: 0483 cost = 0.374977902
Validation Loss: 0.4053356
Epoch: 0484 cost = 0.374056091
Validation Loss: 0.39337343
Epoch: 0485 cost = 0.373139019
Validation Loss: 0.3853348
Epoch: 0486 cost = 0.372225846
Validation Loss: 0.40720227
Epoch: 0487 cost = 0.371316681
Validation Loss: 0.38829488
Epoch: 0488 cost = 0.370412340
Validation Loss: 0.38741907
Epo

Epoch: 0622 cost = 0.279756089
Validation Loss: 0.32402435
Epoch: 0623 cost = 0.279263864
Validation Loss: 0.3135903
Epoch: 0624 cost = 0.278773723
Validation Loss: 0.304335
Epoch: 0625 cost = 0.278285854
Validation Loss: 0.310394
Epoch: 0626 cost = 0.277800448
Validation Loss: 0.30213153
Epoch: 0627 cost = 0.277316988
Validation Loss: 0.3068626
Epoch: 0628 cost = 0.276835710
Validation Loss: 0.29340166
Epoch: 0629 cost = 0.276356672
Validation Loss: 0.302803
Epoch: 0630 cost = 0.275879952
Validation Loss: 0.3056958
Epoch: 0631 cost = 0.275405360
Validation Loss: 0.3081791
Epoch: 0632 cost = 0.274932705
Validation Loss: 0.31688076
Epoch: 0633 cost = 0.274462263
Validation Loss: 0.31760514
Epoch: 0634 cost = 0.273994125
Validation Loss: 0.3077325
Epoch: 0635 cost = 0.273528067
Validation Loss: 0.29823464
Epoch: 0636 cost = 0.273064137
Validation Loss: 0.29737487
Epoch: 0637 cost = 0.272602223
Validation Loss: 0.29883447
Epoch: 0638 cost = 0.272142462
Validation Loss: 0.31248426
Epoch: 0

...

Validation Loss: 0.043695167
Epoch: 4317 cost = 0.025275070
Validation Loss: 0.0434604
Epoch: 4318 cost = 0.025274445
Validation Loss: 0.041936558
Epoch: 4319 cost = 0.025273634
Validation Loss: 0.040724736
Epoch: 4320 cost = 0.025272792
Validation Loss: 0.04087985
Epoch: 4321 cost = 0.025272102
Validation Loss: 0.05423313
Epoch: 4322 cost = 0.025271458
Validation Loss: 0.0498685
Epoch: 4323 cost = 0.025270506
Validation Loss: 0.03614016
Epoch: 4324 cost = 0.025269859
Validation Loss: 0.038289666
Epoch: 4325 cost = 0.025269144
Validation Loss: 0.032453142
Epoch: 4326 cost = 0.025268366
Validation Loss: 0.033957247
Epoch: 4327 cost = 0.025267522
Validation Loss: 0.031929616
Epoch: 4328 cost = 0.025266730
Validation Loss: 0.03195178
Epoch: 4329 cost = 0.025266134
Validation Loss: 0.04324653
Epoch: 4330 cost = 0.025265409
Validation Loss: 0.030581934
Epoch: 4331 cost = 0.025264689
Validation Loss: 0.056380592
Epoch: 4332 cost = 0.025263905
Validation Loss: 0.0789303
Epoch: 4333 cost = 0.0

Validation Loss: 0.032743946
Epoch: 4455 cost = 0.025174546
Validation Loss: 0.039735977
Epoch: 4456 cost = 0.025173814
Validation Loss: 0.03468701
Epoch: 4457 cost = 0.025173141
Validation Loss: 0.0324056
Epoch: 4458 cost = 0.025172449
Validation Loss: 0.0385624
Epoch: 4459 cost = 0.025171581
Validation Loss: 0.044350114
Epoch: 4460 cost = 0.025171050
Validation Loss: 0.043767035
Epoch: 4461 cost = 0.025170450
Validation Loss: 0.037980363
Epoch: 4462 cost = 0.025169690
Validation Loss: 0.034621414
Epoch: 4463 cost = 0.025168960
Validation Loss: 0.032770563
Epoch: 4464 cost = 0.025168156
Validation Loss: 0.036965545
Epoch: 4465 cost = 0.025167428
Validation Loss: 0.034840215
Epoch: 4466 cost = 0.025166830
Validation Loss: 0.04307867
Epoch: 4467 cost = 0.025166035
Validation Loss: 0.037939873
Epoch: 4468 cost = 0.025165382
Validation Loss: 0.037869956
Epoch: 4469 cost = 0.025164603
Validation Loss: 0.05988462
Epoch: 4470 cost = 0.025163986
Validation Loss: 0.05588929
Epoch: 4471 cost = 

Epoch: 4603 cost = 0.025072047
Validation Loss: 0.034334984
Epoch: 4604 cost = 0.025071343
Validation Loss: 0.034910075
Epoch: 4605 cost = 0.025070752
Validation Loss: 0.032554176
Epoch: 4606 cost = 0.025070035
Validation Loss: 0.039803453
Epoch: 4607 cost = 0.025069389
Validation Loss: 0.04457561
Epoch: 4608 cost = 0.025068715
Validation Loss: 0.033417456
Epoch: 4609 cost = 0.025067935
Validation Loss: 0.03127051
Epoch: 4610 cost = 0.025067178
Validation Loss: 0.032386336
Epoch: 4611 cost = 0.025066609
Validation Loss: 0.035542313
Epoch: 4612 cost = 0.025065905
Validation Loss: 0.031970415
Epoch: 4613 cost = 0.025065425
Validation Loss: 0.044569448
Epoch: 4614 cost = 0.025064631
Validation Loss: 0.05061926
Epoch: 4615 cost = 0.025063947
Validation Loss: 0.050269768
Epoch: 4616 cost = 0.025063374
Validation Loss: 0.06083179
Epoch: 4617 cost = 0.025062684
Validation Loss: 0.06551778
Epoch: 4618 cost = 0.025061952
Validation Loss: 0.04814468
Epoch: 4619 cost = 0.025061306
Validation Loss

Validation Loss: 0.044898804
Epoch: 4741 cost = 0.024981528
Validation Loss: 0.033173665
Epoch: 4742 cost = 0.024980889
Validation Loss: 0.035273355
Epoch: 4743 cost = 0.024980289
Validation Loss: 0.032798097
Epoch: 4744 cost = 0.024979611
Validation Loss: 0.032979354
Epoch: 4745 cost = 0.024978967
Validation Loss: 0.031375054
Epoch: 4746 cost = 0.024978337
Validation Loss: 0.035634622
Epoch: 4747 cost = 0.024977754
Validation Loss: 0.035618544
Epoch: 4748 cost = 0.024977046
Validation Loss: 0.041644745
Epoch: 4749 cost = 0.024976431
Validation Loss: 0.03963967
Epoch: 4750 cost = 0.024975773
Validation Loss: 0.044012677
Epoch: 4751 cost = 0.024975177
Validation Loss: 0.03621865
Epoch: 4752 cost = 0.024974480
Validation Loss: 0.03323509
Epoch: 4753 cost = 0.024973944
Validation Loss: 0.038041636
Epoch: 4754 cost = 0.024973289
Validation Loss: 0.032988176
Epoch: 4755 cost = 0.024972484
Validation Loss: 0.03671767
Epoch: 4756 cost = 0.024971818
Validation Loss: 0.047191948
Epoch: 4757 cos

Epoch: 4898 cost = 0.024884642
Validation Loss: 0.03128984
Epoch: 4899 cost = 0.024884051
Validation Loss: 0.030543828
Epoch: 4900 cost = 0.024883304
Validation Loss: 0.034457203
Epoch: 4901 cost = 0.024882760
Validation Loss: 0.031529468
Epoch: 4902 cost = 0.024882211
Validation Loss: 0.03216557
Epoch: 4903 cost = 0.024881487
Validation Loss: 0.032005284
Epoch: 4904 cost = 0.024880865
Validation Loss: 0.036939565
Epoch: 4905 cost = 0.024880411
Validation Loss: 0.03592258
Epoch: 4906 cost = 0.024879707
Validation Loss: 0.03380484
Epoch: 4907 cost = 0.024879199
Validation Loss: 0.033786304
Epoch: 4908 cost = 0.024878535
Validation Loss: 0.033227976
Epoch: 4909 cost = 0.024877979
Validation Loss: 0.032391023
Epoch: 4910 cost = 0.024877381
Validation Loss: 0.04586852
Epoch: 4911 cost = 0.024876911
Validation Loss: 0.036720447
Epoch: 4912 cost = 0.024876220
Validation Loss: 0.033146564
Epoch: 4913 cost = 0.024875560
Validation Loss: 0.03546764
Epoch: 4914 cost = 0.024875041
Validation Loss

In [3]:
import csv
with open('C:/Users/jira_/Desktop/new_stresult22-15-22.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))